# Benchmarking Snapshotting / Altering the system and Running a Simulation

## First order of businees, you will need to have all the dependencies, the jupyter notebook is using.
* You can install a viable Virtulenvironment into the Jupyter kernel

## Boiler plate code:
#### I have made a simple interface, as I have thought that I will need to run many datapoints.

In [1]:
import pandas
from Utilities.SimulationInterface import SimulationInterface
from Utilities.Benchmarking import Benchmarking

#### Setup of a simple run with 10 000 particles
* I am setting the sigma here intentionally, as this is the variable controlling the ideal distance of molecules
* I am also setting the epsilon as this tells us the depth of the potential energy well

In [2]:
simulationObject = SimulationInterface(numberOfParticles = 10000, LJsigma = 0.001, LJepsilon = 0.001)
simulationObject.RunSetup()
simulationObject.RunSimulation(runLength=600)

HOOMD-blue 2.9.6 DOUBLE HPMC_MIXED TBB SSE SSE2 SSE3 
Compiled: 03/17/2021
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, J Glaser, and S C Glotzer. "HOOMD-blue: A Python package for
  high-performance molecular dynamics and hard particle Monte Carlo
  simulations", Computational Materials Science 173 (2020) 109363
-----
HOOMD-blue is running on the CPU
notice(2): Group "all" created containing 10648 particles
notice(2): integrate.langevin/bd is using specified gamma values
notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 10648
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: no
** starting run **
Time 00:00:04 | Step 600 / 600 | TPS 135.584 | ETA 00:00:00
Average TPS: 135.528
---------
-- Neighborlist stats:
24 normal updates / 6 forced updates / 0 dangerous updates
n_neigh_mi

#### Collectring the data from the benchmarks into an empty Dictionary

In [3]:
benchmarkDictionary = {}

#### Benchmarking - Taking a Snapshot and restoring the hoomdSystemsObject

In [4]:
avgTime = Benchmarking.BenchmarkSnapshot(simulationObject.TakeSnapshotNRestore, 10)
print("On average it took: {} seconds!".format(avgTime))
benchmarkDictionary["SnapshotNRestore"] = [avgTime]

0. Run
1. Run
2. Run
3. Run
4. Run
5. Run
6. Run
7. Run
8. Run
9. Run
On average it took: 0.00834641456604004 seconds!


#### Benchmarking - Taking a Snapshot > Altering It and restoring the hoomdSystemsObject
* Interesingly, if you Snapshot > Alter > Restore many times, without running it for at least a step, hoomd Throws an error at you, that you have particles without position, even if you set them manually.

In [5]:
def benchmarkRunSnapshotNAlterNRun():
	simulationObject.TakeSnapshotAlterNRestore()
	simulationObject.RunSimulation(1)

avgTime = Benchmarking.BenchmarkSnapshot(benchmarkRunSnapshotNAlterNRun, 10)
print("On average it took: {} seconds!".format(avgTime))
benchmarkDictionary["AlterNRestore"] = [avgTime]

0. Run
notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 10649
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: no
** starting run **
Time 00:00:05 | Step 601 / 601 | TPS 12.0635 | ETA 00:00:00
Average TPS: 11.9041
---------
-- Neighborlist stats:
0 normal updates / 1 forced updates / 0 dangerous updates
n_neigh_min: 0 / n_neigh_max: 36 / n_neigh_avg: 15.0315
shortest rebuild period: 100
-- Cell list stats:
Dimension: 11, 11, 11
n_min    : 3 / n_max: 15 / n_avg: 8.00075
** run complete **
1. Run
notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 10650
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: no
** starting run **
Time 00:00:05 | Step 602 / 602 | TPS 24.8429 | ETA 00:00:00
Average TPS: 24.1639
---------
-- Neighborlist stats:
0 normal updates / 0 force

#### Benchmarking a 10 mins run (600 seconds, if a timestep is 1 second)

In [6]:
avgTime = Benchmarking.BenchmarkSimulation(simulationObject.RunSimulation, 600, 10)
print("On average it took: {} seconds!".format(avgTime))
benchmarkDictionary["Run10Mins"] = [avgTime]

** starting run **
Time 00:00:14 | Step 1210 / 1210 | TPS 166.897 | ETA 00:00:00
Average TPS: 166.842
---------
-- Neighborlist stats:
26 normal updates / 7 forced updates / 0 dangerous updates
n_neigh_min: 0 / n_neigh_max: 42 / n_neigh_avg: 15.162
shortest rebuild period: 17
-- Cell list stats:
Dimension: 11, 11, 11
n_min    : 1 / n_max: 18 / n_avg: 8.00751
** run complete **
** starting run **
Time 00:00:17 | Step 1810 / 1810 | TPS 192.346 | ETA 00:00:00
Average TPS: 192.284
---------
-- Neighborlist stats:
27 normal updates / 7 forced updates / 0 dangerous updates
n_neigh_min: 0 / n_neigh_max: 45 / n_neigh_avg: 15.2981
shortest rebuild period: 17
-- Cell list stats:
Dimension: 11, 11, 11
n_min    : 1 / n_max: 18 / n_avg: 8.00751
** run complete **


### Showing the benchmarks next to each other.

In [ ]:
BenchmarkRun = pandas.DataFrame.from_dict(benchmarkDictionary, orient="columns")
print(BenchmarkRun)

   SnapshotNRestore  AlterNRestore  Run10Mins
0          0.007974       0.060351   1.882833
